In [7]:
!git clone https://github.com/daffirds/RobloxGameChatAnalysis.git

fatal: destination path 'RobloxGameChatAnalysis' already exists and is not an empty directory.


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
# --- PERUBAHAN 1: Ganti kelas model dari Bert ke Roberta ---
from transformers import RobertaTokenizer, TFRobertaModel
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Bidirectional, GRU, AdditiveAttention, Lambda, LayerNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import re

In [2]:
# Load the dataset
file_path = '/content/RobloxGameChatAnalysis/indonesian_chat.csv'
df = pd.read_csv(file_path)
df['label'] = df['label'].astype(str)

In [5]:
def clean_chat_text(text):
    text = text.replace('\\n', ' ').replace('\n', ' ')
    sentence = re.findall(r'(\\x[0-9a-f]{2})|([a-z0-9]+)', text.lower())
    sentence_list = [item for sublist in sentence for item in sublist if item]
    return ' '.join(sentence_list)

df['chat'] = df['chat'].apply(clean_chat_text)

In [6]:
# Encode labels
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

X_text = df['chat'].values
y = df['label'].values
num_classes = len(np.unique(y))

In [8]:
# Load BERT tokenizer and model
MODEL_NAME = 'flax-community/indonesian-roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
roberta_model = TFRobertaModel.from_pretrained(MODEL_NAME)

max_len = 128

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'roberta.embeddings.position_ids', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: 

In [9]:
# Tokenize dengan RoBERTa tokenizer
def encode_texts(texts):
    return tokenizer(
        list(texts),
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors='np'
    )

encodings = encode_texts(X_text)
X_input_ids = encodings['input_ids']
X_attention_mask = encodings['attention_mask']

In [10]:
# Split the data
train_idx, val_idx = train_test_split(
    np.arange(len(X_text)), test_size=0.2, stratify=y, random_state=42
)

X_train_ids = X_input_ids[train_idx]
X_val_ids = X_input_ids[val_idx]
X_train_mask = X_attention_mask[train_idx]
X_val_mask = X_attention_mask[val_idx]
y_train = y[train_idx]
y_val = y[val_idx]

In [12]:
# Build RoBERTa + Stacked BiGRU + Attention model
def build_model():
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name='input_ids')
    attention_mask = Input(shape=(max_len,), dtype=tf.int32, name='attention_mask')

    # --- PERUBAHAN 3: Gunakan model RoBERTa ---
    # Gunakan Lambda untuk memastikan kompatibilitas
    roberta_output = Lambda(lambda x: roberta_model(x[0], attention_mask=x[1])[0], output_shape=(max_len, 768))([input_ids, attention_mask])

    x = Bidirectional(GRU(64, return_sequences=True))(roberta_output)
    x = LayerNormalization()(x)
    x = Dropout(0.3)(x)

    x = Bidirectional(GRU(32, return_sequences=True))(x)
    x = LayerNormalization()(x)
    x = Dropout(0.3)(x)

    attention_out = AdditiveAttention()([x, x])
    x = Dropout(0.3)(attention_out)

    x = Bidirectional(GRU(32))(x)
    x = Dropout(0.3)(x)

    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=[input_ids, attention_mask], outputs=output)
    model.compile(optimizer=Adam(learning_rate=2e-5, clipnorm=1.0), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Initialize the model
model = build_model()

In [ ]:
# Train the model
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, min_lr=1e-6)
]

history = model.fit(
    [X_train_ids, X_train_mask],
    y_train,
    validation_data=([X_val_ids, X_val_mask], y_val),
    epochs=10,
    batch_size=16,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/10
1071/1071 ━━━━━━━━━━━━━━━━━━━━ 105s 91ms/step - accuracy: 0.3198 - loss: 1.3721 - val_accuracy: 0.5661 - val_loss: 1.2075 - learning_rate: 1.0000e-04
Epoch 2/10
1071/1071 ━━━━━━━━━━━━━━━━━━━━ 141s 90ms/step - accuracy: 0.6182 - loss: 1.0498 - val_accuracy: 0.6950 - val_loss: 0.8065 - learning_rate: 1.0000e-04
Epoch 3/10
1071/1071 ━━━━━━━━━━━━━━━━━━━━ 144s 92ms/step - accuracy: 0.7489 - loss: 0.6836 - val_accuracy: 0.7450 - val_loss: 0.6769 - learning_rate: 1.0000e-04
Epoch 4/10


In [ ]:
# Extract features from GRU model
feature_model = Model(inputs=gru_model.input, outputs=gru_model.output)
X_train_features = feature_model.predict(X_train)
X_val_features = feature_model.predict(X_val)

In [ ]:
# Train Random Forest on extracted features
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_features, y_train)

y_pred = rf.predict(X_val_features)

In [ ]:
# Evaluation
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred, average='weighted')
recall = recall_score(y_val, y_pred, average='weighted')
f1 = f1_score(y_val, y_pred, average='weighted')

print("\nEvaluation Results:")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")
print(f"  F1-Score: {f1:.4f}")

cm = confusion_matrix(y_val, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()